In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-06-16 23:27:29.744746: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
data = pd.read_csv("/Users/devrajjajoo/Downloads/fer2013.csv")


In [9]:
X = []
y = []
for index, row in data.iterrows():
    pixel_values = row['pixels'].split(' ')
    pixel_values = np.array(pixel_values, dtype='float32')


    if pixel_values.shape[0] == 2304:
        image = pixel_values.reshape(48, 48, 1)
        X.append(image)
        y.append(row['emotion'])
    else:
        print(f"Skipping row {index} due to unexpected pixel values shape: {pixel_values.shape}")

X = np.array(X) / 255.0
y = np.array(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

In [12]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [13]:
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
history = model.fit(datagen.flow(X_train.reshape(-1, 48, 48, 1), y_train, batch_size=64),
                    steps_per_epoch=len(X_train) // 64,
                    epochs=50,
                    validation_data=(X_test.reshape(-1, 48, 48, 1), y_test))

Epoch 1/50


2024-06-16 23:28:58.707596: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


448/448 [==============================] - 25s 53ms/step - loss: 1.7958 - accuracy: 0.2580 - val_loss: 1.6868 - val_accuracy: 0.3260
Epoch 2/50
448/448 [==============================] - 23s 51ms/step - loss: 1.6842 - accuracy: 0.3275 - val_loss: 1.5350 - val_accuracy: 0.3975
Epoch 3/50
448/448 [==============================] - 24s 54ms/step - loss: 1.5548 - accuracy: 0.3953 - val_loss: 1.4245 - val_accuracy: 0.4524
Epoch 4/50
448/448 [==============================] - 23s 52ms/step - loss: 1.4673 - accuracy: 0.4355 - val_loss: 1.3122 - val_accuracy: 0.5091
Epoch 5/50
448/448 [==============================] - 24s 54ms/step - loss: 1.4136 - accuracy: 0.4582 - val_loss: 1.2862 - val_accuracy: 0.5132
Epoch 6/50
448/448 [==============================] - 23s 51ms/step - loss: 1.3781 - accuracy: 0.4751 - val_loss: 1.2520 - val_accuracy: 0.5198
Epoch 7/50
448/448 [==============================] - 23s 52ms/step - loss: 1.3465 - accuracy: 0.4883 - val_loss: 1.2281 - val_accuracy: 0.5340
Epo

In [15]:
y_pred = model.predict(X_test.reshape(-1, 48, 48, 1))
y_pred = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

225/225 [==============================] - 2s 9ms/step
Accuracy: 0.5883254388409027


In [16]:
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


In [17]:
cap = cv2.VideoCapture(0)


2024-06-16 23:48:50.249 python[65225:4463952] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


In [18]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (48, 48))
        face_img = face_img.reshape(1, 48, 48, 1) / 255.0

        emotion_idx = np.argmax(model.predict(face_img))
        emotion = emotion_labels[emotion_idx]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Emotion Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step


: 